In [30]:
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

In [31]:
def res_analysis(opt_reg, trn_X, trn_y, tst_X, tst_y):
    """
    Perform regression analysis and return evaluation metrics.
    
    Parameters:
    - opt_reg: Optimized regression model
    - trn_X: Training data features
    - trn_y: Training data labels
    - tst_X: Test data features
    - tst_y: Test data labels
    
    Returns:
    - List of evaluation metrics: [R^2, Pearson correlation, RMSE, Spearman correlation]
    """
    otst_pred = pd.Series(opt_reg.predict(tst_X), dtype=float)
    otrn_pred = pd.Series(opt_reg.predict(trn_X), dtype=float)
    r2 = r2_score(tst_y, otst_pred)
    pear = scipy.stats.pearsonr(tst_y, otst_pred)
    rmse = (mean_squared_error(tst_y, otst_pred)) ** 0.5
    spmn = scipy.stats.spearmanr(tst_y, otst_pred)
    return [r2, pear[0], rmse, spmn[0]]

In [32]:
def find_best_estimator(estimator, parameters, trn_X, trn_y):
    """
    Find the best estimator using GridSearchCV and return it.
    
    Parameters:
    - estimator: Regression estimator
    - parameters: Parameter grid for GridSearchCV
    - trn_X: Training data features
    - trn_y: Training data labels
    
    Returns:
    - Best estimator from GridSearchCV
    """
    grid_search = GridSearchCV(estimator, parameters, scoring='neg_root_mean_squared_error', cv=8)
    model = grid_search.fit(trn_X, trn_y)
    print('Training set R-pearson =', scipy.stats.pearsonr(trn_y, model.predict(trn_X)))
    return model.best_estimator_

In [33]:
def eliminate_high_corr(X_df, corr_thresh):
    """
    Eliminate columns with high correlation and return the reduced DataFrame.
    
    Parameters:
    - X_df: Input DataFrame of features
    - corr_thresh: Threshold for high correlation
    - y: Target variable
    
    Returns:
    - Reduced DataFrame and list of eliminated columns
    """
    X_corr = X_df.corr()
    high_corr = []
    for i in X_df.columns:
        for j in X_df.columns:
            if i != j and abs(X_corr[i][j]) > corr_thresh:
                if (X_df[i].std() <= X_df[j].std()) and i not in high_corr:
                    high_corr.append(i)
                elif (X_df[i].std() > X_df[j].std()) and j not in high_corr:
                    high_corr.append(j)
    print("Columns to drop:", len(high_corr), "\n", high_corr)
    print("Reduced number of columns:", X_df.shape[1] - len(high_corr))
    X_reduced = X_df.drop(high_corr, axis=1)
    return [X_reduced, high_corr]

In [34]:
def fill_pred_dict(dic, spl, nf, y_test, y_pred):
    """
    Fill a prediction dictionary with test and predicted values.
    
    Parameters:
    - dic: Prediction dictionary
    - spl: Split number
    - nf: Number of features
    - y_test: True labels from the test set
    - y_pred: Predicted labels
    
    Returns:
    - Updated prediction dictionary
    """
    dic['test_split-' + str(spl) + '_features-' + str(nf)] = np.array(y_test)
    dic['pred_split-' + str(spl) + '_features-' + str(nf)] = np.array(y_pred)
    return dic    

In [35]:
def plot_and_save(estimator, split, num_features, y_test, y_pred):
    """
    Create a scatter plot of y_test vs y_pred, add a reference line, set plot properties,
    and save the plot as an image file.

    Parameters:
        estimator (str): Name or description of the estimator/model used.
        split (int): Split or fold number.
        num_features (int): Number of features used in the model.
        y_test (array-like): True target values.
        y_pred (array-like): Predicted target values.
    """
    plt.figure(figsize=(8, 8))
    plt.scatter(y_test, y_pred, alpha=0.6, color='blue', label='Actual vs. Predicted')
    plt.plot([-9, -2], [-9, -2], linestyle='--', color='black', label='Perfect Match')
    
    title = f'Split {split} | {num_features} Features | Estimator: {estimator}'
    plt.title(title, fontsize=16)
    plt.xlabel('Actual y_test (kcal/mol)', fontsize=14)
    plt.ylabel('Predicted y_pred (kcal/mol)', fontsize=14)
    plt.legend(loc='upper left', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.5)
    
    save_filename = f'Split{split}_Features{num_features}_{estimator}.png'
    plt.savefig(save_filename)
    
    plt.show()

# Example usage:
# plot_and_save('Random Forest', 1, 10, y_test, y_pred)

In [45]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold

# Constants
VARIANCE_THRESHOLD = 0.01
CORRELATION_THRESHOLD = 0.7

def preprocess_data(df):
    # Remove columns with low variance
    to_drop = df.columns[-1:]
    X = df.drop(to_drop, axis=1)

    # Apply variance threshold and correlation threshold
    vt = VarianceThreshold(VARIANCE_THRESHOLD)
    X0 = pd.DataFrame(vt.fit_transform(X), index=X.index, columns=X.columns[vt.get_support()])
    X1 = eliminate_high_corr(X0, CORRELATION_THRESHOLD)[0]
    return X1

# Load your data into DF0
DF0 = pd.read_excel("DataFrame.xlsx", index_col=0)

# Preprocess the data
X2 = preprocess_data(DF0)
y = DF0.iloc[:,-1]

# Define a list of numbers of features to consider
# num_features = [2, 4, 6, 8, 10, 12, X2.shape[1]]

# Create empty DataFrames for results
result_columns = ['RF', 'GBR', 'SVR', 'LR']
result_df_names = [f'{model}{nf}' for nf in num_features for model in result_columns]

spl_is = [f'split_{i}' for i in range(10)]
feaimpdf = pd.DataFrame(0, index=spl_is, columns=X2.columns)
r2df = pd.DataFrame(index=spl_is, columns=result_df_names)
peardf = pd.DataFrame(index=spl_is, columns=result_df_names)
rmsedf = pd.DataFrame(index=spl_is, columns=result_df_names)
spmndf = pd.DataFrame(index=spl_is, columns=result_df_names)

lin_dict_pred = {}
gbr_dict_pred = {}
svr_dict_pred = {}
rfr_dict_pred = {}

Columns to drop: 1 
 [1]
Reduced number of columns: 1


In [48]:
ntfs = [100,150,200,250]
maxfeafs =['sqrt','log2','auto']
minsmplffs = [1,3,5]
parfs = {'max_features':maxfeafs, 'n_estimators':ntfs, 'min_samples_leaf':minsmplffs}

ne = [100,200,300,400,500]
minsmpleaf = [1,3,5]
mxf =  ['sqrt','log2','auto']
rfpar = {'n_estimators':ne, 'max_features':mxf, 'min_samples_leaf':minsmpleaf} 

gbrne = [100,150,200,250]
gbrpar = {'max_features':mxf, 'n_estimators':gbrne, 'min_samples_leaf':minsmpleaf}

c = np.logspace(-1,1,15)
gam = ['scale','auto']
ker = ['rbf']  
svrpar = {'C':c, 'gamma':gam, 'kernel':ker}

In [50]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
import scipy.stats

# Constants
NUM_SPLITS = 10
RANDOM_SEED = 17061991

def main():
    # Load your data into DF03
    
    #X2 = preprocess_data(DF03)
    
    num_features_to_try = [2, 4, 6, 8, 10, 12, X2.shape[1]]
    result_columns = ['RF', 'GBR', 'SVR', 'LR']

    for spl in range(NUM_SPLITS):
        print('\nCurrently running Split', spl)
        X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.2, random_state=RANDOM_SEED + (43210 * spl))
        
        gbrfs = GradientBoostingRegressor(random_state=RANDOM_SEED + (54321 * spl))
        gbrfs_best = find_best_estimator(gbrfs, parfs, X_train, y_train)
        gbrfs_features = X2.columns[gbrfs_best.feature_importances_.argsort()[::-1][:num_features_to_try[-1]]]
        
        for nf in num_features_to_try:
            print("\nNumber Of Features:", nf)
            top_features = gbr_features[:nf]
            X_trn, X_tst = X_train[top_features], X_test[top_features]
            
            linr = LinearRegression()
            linr_fit = linr.fit(X_trn, y_train)
            lr_r_squared, lr_pearson_r, lr_rmse, lr_spmn = res_analysis(linr_fit, X_trn, y_train, X_tst, y_test)
            
            svr = SVR()
            svr_best = find_best_estimator(svr, svrpar, X_trn, y_train)
            svr_r_squared, svr_pearson_r, svr_rmse, svr_spmn = res_analysis(svr_best, X_trn, y_train, X_tst, y_test)
            
            gbr = GradientBoostingRegressor(random_state=RANDOM_SEED + (65432 * spl))
            gbr_best = find_best_estimator(gbr, gbrpar, X_trn, y_train)
            gbr_r_squared, gbr_pearson_r, gbr_rmse, gbr_spmn = res_analysis(gbr_best, X_trn, y_train, X_tst, y_test)
            
            rfr = RandomForestRegressor(random_state=RANDOM_SEED + (76543 * spl))
            rfr_best = find_best_estimator(rfr, rfpar, X_trn, y_train)
            rfr_r_squared, rfr_pearson_r, rfr_rmse, rfr_spmn = res_analysis(rfr_best, X_trn, y_train, X_tst, y_test)
            
            # Store the results in DataFrames
            
            # Plot and save the results if needed
            # plot_and_save('LR', spl, nf, y_test, linr.predict(X_tst))
            # plot_and_save('SVR', spl, nf, y_test, svr_best.predict(X_tst))
            # plot_and_save('GBR', spl, nf, y_test, gbr_best.predict(X_tst))
            # plot_and_save('RFR', spl, nf, y_test, rfr_best.predict(X_tst))

if __name__ == "__main__":
    main()


Currently running Split 0


ValueError: Cannot have number of splits n_splits=8 greater than the number of samples: n_samples=2.